In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Datensatz laden mit robusteren Parsing-Optionen
# Versuche verschiedene Encoding-Optionen und handle problematische Zeilen
print("Lade Datensatz...")

# Prüfe pandas-Version für Kompatibilität
pandas_version = pd.__version__
print(f"Pandas-Version: {pandas_version}")

# Basis-Parameter für read_csv
base_params = {
    'engine': 'python',  # Python-Engine ist robuster bei komplexen CSV-Dateien
    'quoting': 1,  # QUOTE_ALL
}

# Füge Parameter für problematische Zeilen hinzu (abhängig von pandas-Version)
if pd.__version__ >= '1.3.0':
    base_params['on_bad_lines'] = 'skip'
else:
    base_params['error_bad_lines'] = False
    base_params['warn_bad_lines'] = False

# Versuche verschiedene Encodings
encodings = ['utf-8', 'latin-1', 'cp1252']
df = None

for encoding in encodings:
    try:
        params = base_params.copy()
        params['encoding'] = encoding
        df = pd.read_csv('../data/de_hf_112024.csv', **params)
        print(f"✓ Datei erfolgreich mit {encoding} Encoding geladen")
        break
    except Exception as e:
        print(f"✗ {encoding} fehlgeschlagen: {str(e)[:100]}")
        continue

if df is None:
    raise Exception("Konnte Datei mit keinem der versuchten Encodings laden")

print(f"\nGesamtanzahl der Zeilen: {len(df)}")
print(f"Spalten: {df.columns.tolist()}")
print(f"\nLabel-Verteilung:")
print(df['labels'].value_counts())
print(f"\nErste Zeilen:")
df.head()


Lade Datensatz...
Pandas-Version: 2.3.3
✓ Datei erfolgreich mit utf-8 Encoding geladen

Gesamtanzahl der Zeilen: 50544
Spalten: ['text', 'labels', 'source', 'dataset', 'nb_annotators']

Label-Verteilung:
labels
0.0    44246
1.0     6297
Name: count, dtype: int64

Erste Zeilen:


,text,labels,source,dataset,nb_annotators
0,gleich an die wand stellen und erschiessen..,1.0,Facebook,Bretschneider,2.0
1,nicht dass ich der Grundbotschaft dieses Post'...,0.0,Facebook,Bretschneider,2.0
2,"Das mit dem ""an die Wand stellen und erschiessen""",0.0,Facebook,Bretschneider,2.0
3,"Seit dem ""an die Wand stellen und erschiessen""...",0.0,Facebook,Bretschneider,2.0
4,Ja ja die Kriminelle Heimatpartei FPÖ von Kind...,0.0,Facebook,Bretschneider,2.0


In [6]:
# Daten filtern: Nur Labels 0 oder 1, nur text und labels Spalten
print("Filtere Daten...")
print(f"Ursprüngliche Anzahl Zeilen: {len(df)}")

# Filtere nur Zeilen mit labels 0.0 oder 1.0
df_filtered = df[df['labels'].isin([0.0, 1.0])].copy()

# Behalte nur text und labels Spalten
df_filtered = df_filtered[['text', 'labels']].copy()

# Konvertiere labels zu Integer (0.0 -> 0, 1.0 -> 1)
df_filtered['labels'] = df_filtered['labels'].astype(int)

# Entferne Zeilen mit fehlenden Werten
df_filtered = df_filtered.dropna()

print(f"Gefilterte Anzahl Zeilen: {len(df_filtered)}")
print(f"\nLabel-Verteilung nach Filterung:")
print(df_filtered['labels'].value_counts())
print(f"\nErste Zeilen des gefilterten Datensatzes:")
df_filtered.head()

# Verwende den gefilterten Datensatz für den Split
df = df_filtered


Filtere Daten...
Ursprüngliche Anzahl Zeilen: 50544
Gefilterte Anzahl Zeilen: 50543

Label-Verteilung nach Filterung:
labels
0    44246
1     6297
Name: count, dtype: int64

Erste Zeilen des gefilterten Datensatzes:


In [7]:
# Zufällige Aufteilung: Zuerst 5% für Validation, dann Rest 80/20 für Train/Test
# stratify=df['labels'] sorgt dafür, dass die Label-Verteilung in allen Datensätzen ähnlich ist
print("Teile Datensatz auf...")
print(f"Gesamtanzahl: {len(df)} Zeilen")

# Prüfe ob stratify möglich ist (jede Klasse muss mindestens 2 Samples haben)
label_counts = df['labels'].value_counts()
min_samples = label_counts.min()

if min_samples >= 2:
    # Schritt 1: 5% für Validierungsdatensatz abspalten
    df_temp, val_df = train_test_split(
        df,
        test_size=0.05,
        random_state=42,
        stratify=df['labels']
    )
    
    # Schritt 2: Den Rest (95%) 80/20 in Train und Test aufteilen
    # 80% von 95% = 76% des Gesamtdatensatzes
    # 20% von 95% = 19% des Gesamtdatensatzes
    train_df, test_df = train_test_split(
        df_temp,
        test_size=0.20,  # 20% von den verbleibenden 95%
        random_state=42,
        stratify=df_temp['labels']
    )
    print("✓ Stratified Split durchgeführt")
else:
    # Fallback: Normaler Split ohne Stratification
    df_temp, val_df = train_test_split(df, test_size=0.05, random_state=42)
    train_df, test_df = train_test_split(df_temp, test_size=0.20, random_state=42)
    print("⚠ Normaler Split (ohne Stratification) - eine Klasse hat zu wenige Samples")

print(f"\n✅ Aufteilung abgeschlossen:")
print(f"  Trainingsdatensatz: {len(train_df)} Zeilen ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validierungsdatensatz: {len(val_df)} Zeilen ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Testdatensatz: {len(test_df)} Zeilen ({len(test_df)/len(df)*100:.1f}%)")

print(f"\nLabel-Verteilung im Trainingsdatensatz:")
print(train_df['labels'].value_counts())
print(f"\nLabel-Verteilung im Validierungsdatensatz:")
print(val_df['labels'].value_counts())
print(f"\nLabel-Verteilung im Testdatensatz:")
print(test_df['labels'].value_counts())


Teile Datensatz auf...
Gesamtanzahl: 50543 Zeilen
✓ Stratified Split durchgeführt

✅ Aufteilung abgeschlossen:
  Trainingsdatensatz: 38412 Zeilen (76.0%)
  Validierungsdatensatz: 2528 Zeilen (5.0%)
  Testdatensatz: 9603 Zeilen (19.0%)

Label-Verteilung im Trainingsdatensatz:
labels
0    33626
1     4786
Name: count, dtype: int64

Label-Verteilung im Validierungsdatensatz:
labels
0    2213
1     315
Name: count, dtype: int64

Label-Verteilung im Testdatensatz:
labels
0    8407
1    1196
Name: count, dtype: int64


In [8]:
# Datensätze speichern (nur text und labels Spalten)
train_df.to_csv('../data/de_hf_112024_train.csv', index=False)
val_df.to_csv('../data/de_hf_112024_val.csv', index=False)
test_df.to_csv('../data/de_hf_112024_test.csv', index=False)

print("Datensätze erfolgreich gespeichert:")
print("- ../data/de_hf_112024_train.csv")
print("- ../data/de_hf_112024_val.csv")
print("- ../data/de_hf_112024_test.csv")
print(f"\nGespeicherte Spalten: {train_df.columns.tolist()}")
print(f"\nBeispiel aus Trainingsdatensatz:")
print(train_df.head())

# Zusammenfassung
print("\n" + "="*60)
print("ZUSAMMENFASSUNG")
print("="*60)
print(f"\nGesamtanzahl Zeilen (Original): {len(df)}")
print(f"\nTrainingsdatensatz:")
print(f"  - Anzahl Zeilen: {len(train_df)}")
print(f"  - Label-Verteilung:")
for label, count in train_df['labels'].value_counts().sort_index().items():
    print(f"    Label {label}: {count} ({count/len(train_df)*100:.2f}%)")

print(f"\nValidierungsdatensatz:")
print(f"  - Anzahl Zeilen: {len(val_df)}")
print(f"  - Label-Verteilung:")
for label, count in val_df['labels'].value_counts().sort_index().items():
    print(f"    Label {label}: {count} ({count/len(val_df)*100:.2f}%)")

print(f"\nTestdatensatz:")
print(f"  - Anzahl Zeilen: {len(test_df)}")
print(f"  - Label-Verteilung:")
for label, count in test_df['labels'].value_counts().sort_index().items():
    print(f"    Label {label}: {count} ({count/len(test_df)*100:.2f}%)")

print(f"\nGesamt (Train + Val + Test): {len(train_df) + len(val_df) + len(test_df)} Zeilen")
print("="*60)


Datensätze erfolgreich gespeichert:
- ../data/de_hf_112024_train.csv
- ../data/de_hf_112024_val.csv
- ../data/de_hf_112024_test.csv

Gespeicherte Spalten: ['text', 'labels']

Beispiel aus Trainingsdatensatz:
                                                    text  labels
12999  Wenn es zu kalt ist : Klimawandel Wenn es zu w...       0
23082      Wie wäre es denn mit "Flugblätter" verteilen?       0
18557  Und da kommt schon die nächste Horrormeldung: ...       0
5969   Genauso kam Hitler an die Macht. Merkel & Co. ...       0
536    Worte eines Feigen, Morallosen groß Sprechers....       0

ZUSAMMENFASSUNG

Gesamtanzahl Zeilen (Original): 50543

Trainingsdatensatz:
  - Anzahl Zeilen: 38412
  - Label-Verteilung:
    Label 0: 33626 (87.54%)
    Label 1: 4786 (12.46%)

Validierungsdatensatz:
  - Anzahl Zeilen: 2528
  - Label-Verteilung:
    Label 0: 2213 (87.54%)
    Label 1: 315 (12.46%)

Testdatensatz:
  - Anzahl Zeilen: 9603
  - Label-Verteilung:
    Label 0: 8407 (87.55%)
    Label 1